Given that matrix $L$ is a lower triangular matrix 

$$L = \begin{pmatrix} 
l_{11} & 0 & 0 & 0 & 0 \\
l_{21} & l_{22} & 0 & 0 & 0 \\
l_{31} & l_{32} & l_{33} &\cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & l_{n3} & \cdots & l_{nn}
\end{pmatrix}
$$

let $x$ and $b$ be vectors of length $n$ then for $Lx =b$ solving for $x$ can be done in $O(n^2)$ complexity.


$l_{11}x_1 = b_1$<br>
$l_{21}x_1 + l_{22}x_2 = b_2$<br>
$l_{31}x_1 + l_{32}x_2 + l_{33}x_3 = b_3$<br>
$\vdots\qquad\vdots\qquad\vdots\qquad\vdots$<br>
$l_{i1}x_1 + l_{i2}x_2 + l_{i3}x_3 + \cdots + l_{ii}x_i= b_i$<br>
$\vdots\qquad\vdots\qquad\vdots\qquad\vdots\qquad\vdots$<br>
$l_{n1}x_1 + l_{n2}x_2 + l_{n3}x_3 + \cdots + l_{ni}x_i + \cdots + l_{nn}x_n= b_i$
<br>
<br>
<br>
To derive the formula stat by solving for $x_1$
$$x_1 = \frac{b_1}{l_{11}}$$
<br>
Knowing $x_1$ means we can solve for $x_2$
$$x_2 = \frac{b_2 - l_{21}x_1}{l_{22}}$$
<br>
now $x_3$
$$x_3 = \frac{b_3 - (l_{31}x_1 + l_{32}x_2)}{l_{33}}$$
<br>
Continuing in this fashion to solve for the $x_i$ gives us a simple algorithm to solve for any lower triangular system
$$x_i = \frac{b_i - (l_{i1}x_1 + l_{i2}x_2 + l_{i3}x_3 + \cdots + l_{i(i-1)}x_{(i-1)}}{l_{ii}}$$
<br>
$$ = \Bigg(b_i - \sum^{i-1}_{j=1}l_{ij}x_{j}\Bigg)\Bigg/l_{ii}$$

The two functions below both follow the same algorithm with the only diffrence being that the first one is expermenting with julias ability to use refrences of memory instead of passing by value

In [216]:
function forwardSub(A::Ref, b::Ref)
	x = zeros(size(b[]))
	x[1] = (b[])[1] / (A[])[1,1]
	for i in 2:(size((b[]))[1])
		for j in 1:i-1
			x[i] += (A[])[i,j]*x[j]
		end
		x[i] = ((b[])[i] - x[i])/(A[])[i,i]
	end
	return Ref(x)
end

function forwardSub(A, b)
	x = zeros(size(b))
	x[1] = b[1] / A[1,1]
	for i in 2:size(b)[1]
		for j in 1:i-1
			x[i] += A[i,j]*x[j]
		end
		x[i] = (b[i] - x[i])/A[i,i]
	end
	return x
end

forwardSub (generic function with 2 methods)

Below are the bench marks of the two functions that I have wrote and the thrid is julias built in function

In [217]:
using LinearAlgebra
using BenchmarkTools
n = 1000;
L = LowerTriangular(rand(n,n));
b = rand(n,1);

In [218]:
@benchmark forwardSub(Ref(L),Ref(b))[]

BenchmarkTools.Trial: 
  memory estimate:  7.98 KiB
  allocs estimate:  4
  --------------
  minimum time:     944.817 μs (0.00% GC)
  median time:      1.086 ms (0.00% GC)
  mean time:        1.205 ms (0.30% GC)
  maximum time:     16.381 ms (90.15% GC)
  --------------
  samples:          4125
  evals/sample:     1

In [219]:
@benchmark forwardSub(L,b)

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     831.994 μs (0.00% GC)
  median time:      884.613 μs (0.00% GC)
  mean time:        903.082 μs (0.00% GC)
  maximum time:     1.783 ms (0.00% GC)
  --------------
  samples:          5442
  evals/sample:     1

In [220]:
@benchmark L\b

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     421.005 μs (0.00% GC)
  median time:      453.247 μs (0.00% GC)
  mean time:        506.931 μs (0.00% GC)
  maximum time:     3.895 ms (0.00% GC)
  --------------
  samples:          9605
  evals/sample:     1